In [1]:
import pymongo
import pandas as pd

In [12]:
#konfigurasi koneksi
username = 'hridayanikaruna'
password = 'Bigdata123'
host = 'cluster0.ynqzvyj.mongodb.net'
dbname = 'tripadvisor_sentiment'

# URL koneksi MongoDB Atlas
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

# koneksi ke server MongoDB
client = pymongo.MongoClient(uri)

if client:
    print("Koneksi berhasil terhubung!")

Koneksi berhasil terhubung!


In [3]:
collection_name = 'dataset'
db = client[dbname]
collection = db[collection_name]
documents = collection.find()
documents

In [4]:
data = pd.DataFrame(list(documents))
data

,_id,,title,rating,date_of_stay,review,resort_name
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali
...,...,...,...,...,...,...,...
11323,64583cf7f95018753d35db6a,2366,Random visit but surprisingly very good,40,August 2013,"On our first night at Bali, we randomly visite...",U Paasha Seminyak
11324,64583cf7f95018753d35db70,2372,"A brand new modern hotel, comfortable and central",40,July 2013,We stayed here 3 nights and I have to say It w...,U Paasha Seminyak
11325,64583cf7f95018753d35db82,2390,"Smelly, Noisy, 3 rooms in 24hrs!",20,July 2013,We had seen all the great reviews for this pla...,U Paasha Seminyak
11326,64583cf7f95018753d35db94,2408,Fantastic,50,June 2013,The hotel is great and it is quite new. Locati...,U Paasha Seminyak


In [5]:
import nltk
import regex as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [6]:
def preprocess_text(text):
    # Mengubah ke lowercase
    text = text.lower()
    
    # Menghapus url dan http
    text = re.sub(r'http\S+', '', text)
    
    # Menghapus karakter selain huruf dan angka
    text = re.sub(r'[^\w\s]', '', text)
    
    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(filtered_words)
    
    # Menghapus emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # simbol
                               u"\U0001F680-\U0001F6FF"  # transportasi
                               u"\U0001F1E0-\U0001F1FF"  # bendera negara
                               u"\U00002702-\U000027B0"  # bintang dan simbol
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # Melakukan stemming
    ps = PorterStemmer()
    words = word_tokenize(text)
    stemmed_words = [ps.stem(word) for word in words]
    text = ' '.join(stemmed_words)
    
    # Melakukan tokenisasi
    words = word_tokenize(text)
    
    return [word.lower() for word in words]

In [7]:
data['preprocessing_title'] = data['title'].apply(preprocess_text)

In [8]:
data

,_id,,title,rating,date_of_stay,review,resort_name,preprocessing_title
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali,"[balines, experi, rememb]"
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali,[perfect]
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali,"[magnific, hotel, wonder, staff]"
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali,"[second, stay, legian, 3, month, fantast, hotel]"
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali,"[perfect, everi, way]"
...,...,...,...,...,...,...,...,...
11323,64583cf7f95018753d35db6a,2366,Random visit but surprisingly very good,40,August 2013,"On our first night at Bali, we randomly visite...",U Paasha Seminyak,"[random, visit, surprisingli, good]"
11324,64583cf7f95018753d35db70,2372,"A brand new modern hotel, comfortable and central",40,July 2013,We stayed here 3 nights and I have to say It w...,U Paasha Seminyak,"[brand, new, modern, hotel, comfort, central]"
11325,64583cf7f95018753d35db82,2390,"Smelly, Noisy, 3 rooms in 24hrs!",20,July 2013,We had seen all the great reviews for this pla...,U Paasha Seminyak,"[smelli, noisi, 3, room, 24hr]"
11326,64583cf7f95018753d35db94,2408,Fantastic,50,June 2013,The hotel is great and it is quite new. Locati...,U Paasha Seminyak,[fantast]


In [9]:
data['preprocessing_review'] = data['review'].apply(preprocess_text)

In [10]:
data

,_id,,title,rating,date_of_stay,review,resort_name,preprocessing_title,preprocessing_review
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali,"[balines, experi, rememb]","[wonder, stay, joglo, club, villa, legian, sem..."
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali,[perfect],"[first, moment, arriv, tell, place, special, i..."
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali,"[magnific, hotel, wonder, staff]","[spent, 3, night, villa, 4, club, wonder, time..."
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali,"[second, stay, legian, 3, month, fantast, hotel]","[second, time, legian, 3, month, fb, servic, l..."
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali,"[perfect, everi, way]","[stay, five, night, absolut, love, itth, villa..."
...,...,...,...,...,...,...,...,...,...
11323,64583cf7f95018753d35db6a,2366,Random visit but surprisingly very good,40,August 2013,"On our first night at Bali, we randomly visite...",U Paasha Seminyak,"[random, visit, surprisingli, good]","[first, night, bali, randomli, visit, place, n..."
11324,64583cf7f95018753d35db70,2372,"A brand new modern hotel, comfortable and central",40,July 2013,We stayed here 3 nights and I have to say It w...,U Paasha Seminyak,"[brand, new, modern, hotel, comfort, central]","[stay, 3, night, say, perfect, place, like, ba..."
11325,64583cf7f95018753d35db82,2390,"Smelly, Noisy, 3 rooms in 24hrs!",20,July 2013,We had seen all the great reviews for this pla...,U Paasha Seminyak,"[smelli, noisi, 3, room, 24hr]","[seen, great, review, place, realli, disappoin..."
11326,64583cf7f95018753d35db94,2408,Fantastic,50,June 2013,The hotel is great and it is quite new. Locati...,U Paasha Seminyak,[fantast],"[hotel, great, quit, new, locat, bad, opposit,..."


In [11]:
filename = 'preprocessing_data'
data.to_csv(filename)